`from agents import Agent, EpsilonGreedyAgent, GreedyAgent, GreedyExpectedScoresAgent, RandomAgent`

# Naive agents

## Random agent

First I tried a completely random agent: at each timestep, `RandomAgent` simply chooses one of the actions available to it, uniformly at random. I played 10,000 games with this agent:

`random_agent_scores = RandomAgent().play_games(n_games=10_000, histogram_bins=20)`

**TO-DO**: put in histogram

The mean score for a `RandomAgent` over 10,000 games is less than 50 - that's pretty bad! There is a long tail on the right, with a few scores above 100 being achieved.

In this first pass at implementing the gameplay actions and scoring, I focused on development speed and cleanliness of the design. I deliberatly did not spend time optimizing for runtime speed, and it shows here, as it took several minutes to simulate 10,000 one-player games. I still think this is OK for now; I can optimize later once I start training some deep RL agents.

## Greedy and $\epsilon$-greedy agents

Next let's try a `GreedyAgent`. This agent always selects the action the gives the highest immediate reward.

`greedy_agent_scores = GreedyAgent().play_games(n_games=10_000, histogram_bins=50)`

**TO-DO**: put in histogram

The `GreedyAgent` does substantially better: it achieves mean score above 100, in about 5% of games it scores better than 150, and in a few games it even gets close to 200. 

Still, the performance is not great compared to typcial human gameplay. I'm not a terribly skilled player, and my own threshold for a good game is a score around 200. It's not surprising that this agent is pretty bad: generally you want to look at the dice after your first roll and "roll for" one of the available boxes next. For example, if I roll (2, 3, 4, 6, 6) on my first roll, I might pick up (6, 6) and roll those again in the hopes of getting a small or large straight.

It's pretty clear that a naive $\epsilon$-greedy agent won't do better. Since dice have no memory, there's no advantage of sometimes taking a `RollAction` instead of the best `ScoreAction`; to do better, we'd have to pick our `RollAction` strategically, not at random. Trying out the naive agent confirms that it's not a better approach:

`epsilon_greedy_agent_scores = EpsilonGreedyAgent(epsilon=0.5).play_games(n_games=10_000, histogram_bins=50)`

# Basic strategies

## Smarter greedy agent

Next I'll try to come up with a reasonably good strategy, similar to the one I try to follow when playing in the real world. Roughly speaking, given the five dice values after the first roll I pick an unused box to "go for" and then roll the dice that maximize the probability of hitting the box, i.e. scoring as high as possible in the box, on this turn. For now let's set aside the question of how to pick which box to go for, and focus on figuring out which dice to roll to hit the box.

The function `best_roll_action_for_box` takes values of `roll_values` and `box` and returns the dice to roll, i.e. `roll_action`, that maximize the expected score from that box after the roll (or it can return `None`, in which case the score action is better than any roll actions). Note that I am not computing the total expected score from the `roll_action` on `roll_values`, just the expected score that comes from the particular box selected. The idea is that my agent will simply look at the dice and either take a `ScoreAction` or pick a box to "roll for". The advantage of this is that it places a smaller bound on the number of possible actions than just considering all possible roll actions. However, there may be cases where this is suboptimal: say `roll_action_1` is the best to hit `SmallStraight` and `roll_action_2` is the best to hit `FullHouse`, but there is some other action `roll_action_3` that has higher sum of expected scores for `SmallStraight` and `FullHouse`. I'll neglect such cases for this agent.

To figure out the actions that maximize expected scores in boxes given `roll_values`, I just computed a table of all possibilities. `all_expected_scores_table_by_box(n_rolls_left=1)` gives the expected score in a box for any `roll_values`, `roll_action` pair, and `expected_scores_table()` filters this down to only the best `roll_action` for each `roll_values`, `box` pair. This sets us up to write an agent that then looks at the `roll_values` and available boxes, finds the box to "go for" that gives the highest expected score, and then rolls the dice (or takes the `ScoreAction`) that has this expected score.

This table only looks ahead one roll, but in some cases there may be an action with higher expected score if two more rolls are left in the turn. I computed a table of these actions as well: `all_expected_scores_table_by_box(n_rolls_left=2)`. Here's the result when I only look ahead by one roll:

`greedy_expected_scores_agent_scores = GreedyExpectedScoresAgent().play_games(n_games=10_000, histogram_bins=50)`

![greedy_expected_scores_lookahead_1_roll_agent](greedy_expected_scores_lookahead_1_roll_agent.png)

This agent does much better. In 10,000 games, it got a median score of 202 and mean score of 207, and it scored over 250 over 10% of the time. Somewhat surprisingly, looking ahead by two rolls resulted in slightly worse performance:

![greedy_expected_scores_lookahead_2_rolls_agent](greedy_expected_scores_lookahead_2_rolls_agent.png)

The median score was 198 and the mean score was 201. I definitely should check whether I made a mistake in computing the tables, but intuitively it looks like the lookahead-2 agent sides toward slightly less risky behavior; it has fewer games with score above 300.

The distribution has two prominent peaks, which I believe is due to getting/not getting the upper section bonus. The bonus was not taken into account at all in the agent's strategy. I will try to correct this by increasing the expected score values used for the upper section. The bonus gives a score boost of about 15%, so I'll try using that factor in the next section.

## Adjusting for the upper section bonus

Unfortunately this alteration didn't give any improvement.

![upper_section_multiplier](upper_section_multiplier.png)

Thinking more about it, this alteration needs to be a little more sophisticated. Let's use Sixes as an example. I am fairly neutral about getting 3 Sixes; it's "on track" for the bonus but I'd prefer to have more to make up for bad upper box scores later. I consider 4 Sixes very positive, and 0, 1, or 2 negative. I would guess that updating the rewards in this kind of way will make the agent perform better.

Another thing that comes to mind is that this agent is too greedy in its use of the `Chance` box. Typically I hold off on using this box until I really need it. I could try diminishing its reward value by some factor.

Given some `roll_values`, I could also compute, for each roll action, the sum of the expected scores for all the available boxes and use that as the expected reward in choosing the best action. Instead of "going for" a box, this agent would look at all possible box contributions for all available roll actions.

Finally, I think the true expected scores depend on whether it's the first or second re-roll. The scores I already computed are valid for the second re-roll, but for the first re-roll there might be situations where one choice is expected to lead to a better outcome in two rolls than some other choice would in just one.

All that said, I think it's time to start training an RL agent. I now have a reasonable baseline in `GreedyExpectedScoresAgent`. I could implement more handcrafted strategies along the lines of the above, but I think it's a better idea to try to create an RL agent that automatically figures out better strategies.